In [ ]:
!pip install ase

In [ ]:
import numpy as np
import torch as th
import ase
from ase.cluster.cubic import FaceCenteredCubic
import torch.nn.functional as F


In [ ]:
def atomcell(layers):
  surfaces = [(1, 0, 0), (1, 1, 0), (1, 1, 1)]
  lc = 3.61000
  # make a copper nanoparticle, you can change the layers to make it smaller or larger
  atoms = FaceCenteredCubic('Cu', surfaces,layers, latticeconstant=lc)

  coords = atoms.positions.copy().astype(np.float32)

  coords -= coords.min(0)
  coords -= coords.max(0) / 2


In [ ]:

def affine_matrix_3D(phi, theta, psi, translation):

    cph = th.cos(phi)
    sph = th.sin(phi)
    cth = th.cos(theta)
    sth = th.sin(theta)
    cps = th.cos(psi)
    sps = th.sin(psi)
    line1 = th.stack([cph * cps - sph * cth * sps, -cph *
                     sps - sph * cth * cps, sph * sth, translation[0]], 1)
    line2 = th.stack([sph * cps + cph * cth * sps, -sph *
                     sps + cph * cth * cps, -cph * sth, translation[1]], 1)
    line3 = th.stack([sth * sps + 0 * cph, sth * cps + 0 * cph, cth + 0 * (cph + cps), th.zeros_like(translation[1])],
                     1)
    R = th.stack([line1, line2, line3], 1)
    return R


In [ ]:

def exponential_integral_function(d, c, a):
    #     f(x)= ae ^ {- \frac {(x-b)^2} {2c^2}}
    # a	=	height of the curve's peak
    # b	=	the position of the center of the peak

    # c	=	the standard deviation
    # f(x)	=	function of x
    # e	=	Euler's number
    # x	=	integer
    return np.pi * th.abs(a) * th.exp(-d**2/c**2)


In [ ]:
def ray_transform(vol, phi_rad, theta_rad, psi_rad, translation):
    n_theta = phi_rad.shape[0]
    R = affine_matrix_3D(phi_rad, theta_rad, psi_rad, translation)
    out_size = (n_theta, 1, vol.shape[2], vol.shape[3], vol.shape[4])
    grid = F.affine_grid(R, out_size)
    out = F.grid_sample(vol.expand(
        n_theta, 1, vol.shape[2], vol.shape[3], vol.shape[4]), grid)
    # print(out.shape)
    # out is (N_batch, channels, Z, Y, X)
    sino = th.sum(out, 3)
    return sino


In [ ]:
def project(points, phi, theta, psi, translation, param, FOV, image_pixels, integral_function=exponential_integral_function):

    device="cuda"
    M = image_pixels
    w0 = th.linspace(-FOV[1]/2, FOV[1]/2, M, device=device).float()
    w1 = th.linspace(-FOV[2]/2, FOV[2]/2, M, device=device).float()
    yy, xx = th.meshgrid(w0, w1)

    pixel_intensity=th.zeros(phi.shape[0],M,M,device=device)

    rays_start = th.zeros((4096, 3), device=device)
    rays_end = th.zeros((4096, 3), device=device)

    # z, y, x
    rays_start[:, 0] = 0
    rays_start[:, 1] = yy.ravel()
    rays_start[:, 2] = xx.ravel()

    rays_end[:, 0] = FOV[0]
    rays_end[:, 1] = yy.ravel()
    rays_end[:, 2] = xx.ravel()

    rays = rays_end - rays_start

    for id,_ in enumerate(phi):
        R = affine_matrix_3D(phi[id:id+1], theta[id:id+1], psi[id:id+1],translation[:,id:id+1]).to(device)
        point_cloud_rot = points @ R.squeeze()[:, :-1]

        r_in_ray_system = point_cloud_rot[:,None, :] - rays_start[None, :, :]

        ray_expanded = rays[None, :, :].expand_as(r_in_ray_system)

        proj_onto_ray_part1 = th.cross(r_in_ray_system, ray_expanded,dim=-1)
        proj_onto_ray_part2=th.norm(ray_expanded, dim=-1)[:, :, None]

        proj_onto_ray=proj_onto_ray_part1/proj_onto_ray_part2

        # n_theta, n_atom, M * M
        distance = th.norm(proj_onto_ray, dim=-1)
        # n_theta, n_atom, M * M
        per_atom_intensity = integral_function(distance, param[0], param[1])
        # n_theta, M, M
        pixel_intensity[id] = th.sum(per_atom_intensity, 0).view(M, M)

    return pixel_intensity
